# Data Merging

In [ ]:
# importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
profiles = pd.read_csv('nfl_draft_profiles.csv')
prospects = pd.read_csv('nfl_draft_prospects.csv')
college = pd.read_csv('college_statistics.csv')
print(college)

In [ ]:
qb = college[college['pos_abbr'] == 'QB']
print(qb['player_name'].unique())
print(qb[qb['player_name']=='Patrick Mahomes'])

In [ ]:
college = college.dropna(subset=['player_name', 'value'])

all_stars = college.groupby(['player_id', 'player_name','alt_player_id'])['all_star'].sum().reset_index().rename(columns={'all_star': 'all_star_count'})
college_career = college.groupby(['player_id', 'alt_player_id','player_name','statistic'])['value'].sum().reset_index()
college_stats = college_career.pivot(index=['player_name','player_id','alt_player_id'], columns='statistic', values='value')

college_stats = pd.merge(college_stats, all_stars, on = ['player_name','player_id', 'alt_player_id'], how = 'left')
print(college_stats)

In [ ]:
data = pd.merge(profiles, prospects, on = 'player_id', suffixes = ('', '_y'))
data.drop(data.filter(regex='_y$').columns, axis=1, inplace=True)
data = data.dropna(subset = ['player_id'])
print(data)

In [ ]:
df = pd.merge(data, college_stats, on = ['player_name', 'player_id'], how = 'left')
df = df.dropna(subset = ['guid'])

# QB SPECIFICS

In [ ]:
qbr = pd.read_csv('college_qbr.csv')

In [ ]:
average_columns = [
    col for col in qbr.columns if col not in ['guid', 'player_name', 'season', 'points_added', 'age']
]

qbr_summary = qbr.groupby(['guid', 'player_name']).agg({col: 'mean' for col in average_columns},).join(
    qbr.groupby(['guid', 'player_name'])['points_added'].sum().rename('career_sum_points_added')).reset_index()

qbr_summary.rename(columns={col: f"career_avg_{col}" for col in average_columns}, inplace=True)
qbr_summary['career_avg_qb_plays'] = qbr_summary['career_avg_qb_plays'].astype(int)

In [ ]:
dataset = pd.merge(df, qbr_summary, on = ['player_name', 'guid'], how = 'left')
print(dataset)

# Cleaning UP

In [ ]:
dataset = dataset.rename(columns = {'alt_player_id_x':'alt_player_id'})
dataset = dataset.drop(columns = ['position','player_image', 'link', 'school_logo', 'school', 'school_name', 'traded', 'trade_note', 'team',
                                 'team_logo_espn', 'alt_player_id_y'])
print(dataset)

In [ ]:
dataset = dataset.copy()
dataset['scouting'] = dataset[['text1', 'text2', 'text3', 'text4']].apply(
    lambda row: ' '.join(row.dropna()), axis=1
)
dataset = dataset.drop(columns = ['text1', 'text2', 'text3', 'text4', 'Total Points', 'all_star_count'])

In [ ]:
print(dataset['draft_year'].unique())

In [ ]:
dataset = dataset[(dataset['draft_year'] >= 2012) & (dataset['draft_year']<= 2020)]
print(dataset['draft_year'])

# FIXING QB vals`

In [ ]:
scrap_qb = pd.read_csv('complete_qb.csv')

In [ ]:
print(scrap_qb.columns)
scrap_qb = scrap_qb.drop(columns = ['year_id', 'pos'])
print(scrap_qb[scrap_qb['player_name'] == 'Kyler Murray']['pass_td'])

In [ ]:
print(dataset.columns)
print(len(dataset))
print(dataset[dataset['player_name'] == 'Kyler Murray'])

In [ ]:
qb_mapping = {
    'Completion Percentage':'pass_cmp_pct',
    'Completions':'pass_cmp',
    'Interceptions':'pass_int',
    'Passer Rating':'pass_rating',
    'Passing Attempts':'pass_att',
    'Passing Touchdowns':'pass_td',
    'Passing Yards':'pass_yds',
    'Receiving Touchdowns':'rec_td',
    'Receiving Yards':'rec_yds',
    'Receptions':'rec',
    'Rushing Attempts':'rush_att',
    'Rushing Touchdowns':'rush_td',
    'Rushing Yards':'rush_yds',
    'Total Touchdowns':'scrim_td',
    'Yards Per Pass Attempt':'pass_yds_per_att',
    'Yards Per Reception':'rec_yds_per_rec',
    'Yards Per Rush Attempt':'rush_yds_per_att'
}

In [ ]:
qb_only_relevant = scrap_qb[['player_name'] + list(qb_mapping.values())]

merged_df = pd.merge(
    dataset,
    qb_only_relevant,
    on='player_name',
    how='left'
)

for multi_col, qb_col in qb_mapping.items():
    merged_df[multi_col] = merged_df[multi_col].combine_first(merged_df[qb_col])

merged_df = merged_df.drop(columns=list(qb_mapping.values()))

print(merged_df)

## Incorporating Combine Stats

In [ ]:
def fuzzy_match(row, df_to_match, column_name):
    name = row['player_name']
    matches = process.extract(name, df_to_match[column_name], scorer=fuzz.partial_ratio)
    best_match, score, _ = matches[0]
    return best_match if score >= 80 else None

In [ ]:
combine = pd.read_csv('complete_combine.csv')
print(combine['Year'].unique())
combine = combine.drop(columns = ['School', 'College', 'Ht', 'Wt', 'Drafted', 'Unnamed: 0', 'Year'])
print(combine)

In [ ]:
merged_df['matched_name'] = merged_df.apply(fuzzy_match, df_to_match=combine, column_name='Player', axis=1)
merged_df = pd.merge(merged_df, combine, left_on=['matched_name', 'pos_abbr'], right_on=['Player', 'Pos'], how = 'left')
print(merged_df[['player_name', 'matched_name']])

In [ ]:
print(merged_df.columns)

In [ ]:
merged_df = merged_df.drop(columns = ['matched_name', 'Player', 'Pos', 'guid', 'alt_player_id', 'school_abbr', 'pos_rk', 'ovr_rk', 'grade',
                                      'pick', 'overall', 'round', 'team_abbr'])
merged_df.to_csv('complete_nan.csv')